# Getting the data

In [1]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

url = f'{github_url}?raw=1'

In [2]:
import pandas as pd

df = pd.read_csv(url)

In [3]:
df = df.iloc[:300]
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


# Q1. Getting the embeddings model

In [4]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
answer_llm = df.iloc[0]['answer_llm']
answer_orig = df.iloc[0]['answer_orig']

In [15]:
print(answer_llm)

You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).


In [16]:
print(answer_orig)

Machine Learning Zoomcamp FAQ
The purpose of this document is to capture frequently asked technical questions.
We did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:
Data Engineering Zoomcamp FAQ
In the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo
work


In [11]:
v_llm = embedding_model.encode(answer_llm)

v_llm[0]

-0.42244688

In [12]:
v_orig = embedding_model.encode(answer_orig)

cosine_score = v_llm.dot(v_orig)
print(cosine_score)

17.515993


# Q2. Computing the dot product

In [17]:
def compute_similarity(record):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']

    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    return v_llm.dot(v_orig)

In [18]:
from tqdm.auto import tqdm

evaluations = []

result_gpt4omini = df.to_dict(orient='records')

for record in tqdm(result_gpt4omini):
    sim = compute_similarity(record)
    evaluations.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [19]:
df['cosine'] = evaluations
df['cosine'].describe()

count    300.000000
mean      27.495996
std        6.384744
min        4.547922
25%       24.307845
50%       28.336864
75%       31.674307
max       39.476017
Name: cosine, dtype: float64

# Q3. Computing the cosine

In [20]:
import numpy as np

def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [21]:
def compute_similarity(record, normalize=True):
    answer_orig = record['answer_orig']
    answer_llm = record['answer_llm']
    
    v_llm = embedding_model.encode(answer_llm)
    v_orig = embedding_model.encode(answer_orig)

    if normalize:
        v_llm = normalize_vector(v_llm)
        v_orig = normalize_vector(v_orig)

    return v_llm.dot(v_orig)

In [22]:
from tqdm.auto import tqdm

evaluations = []

result_gpt4omini = df.to_dict(orient='records')

for record in tqdm(result_gpt4omini):
    sim = compute_similarity(record, normalize=True)
    evaluations.append(sim)

  0%|          | 0/300 [00:00<?, ?it/s]

In [23]:
df['cosine'] = evaluations
df['cosine'].describe()

count    300.000000
mean       0.728392
std        0.157755
min        0.125357
25%        0.651273
50%        0.763761
75%        0.836235
max        0.958796
Name: cosine, dtype: float64

# Q4. Rouge

In [24]:
df.head()

,answer_llm,answer_orig,document,question,course,cosine
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.506754
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.388549
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.718599
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0.337266
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.521792


In [27]:
answer_llm = df.iloc[10]['answer_llm']
answer_orig = df.iloc[10]['answer_orig']

In [28]:
from rouge import Rouge

rouge_score = Rouge()

scores = rouge_score.get_scores(answer_llm, answer_orig)

In [29]:
scores

[{'rouge-1': {'r': 0.45454545454545453,
   'p': 0.45454545454545453,
   'f': 0.45454544954545456},
  'rouge-2': {'r': 0.21621621621621623,
   'p': 0.21621621621621623,
   'f': 0.21621621121621637},
  'rouge-l': {'r': 0.3939393939393939,
   'p': 0.3939393939393939,
   'f': 0.393939388939394}}]

In [31]:
scores[0]

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [32]:
scores[0]['rouge-1']['f']

0.45454544954545456

# Q5. Average rouge score

In [34]:
rouge_1 = scores[0]['rouge-1']['f']
rouge_2 = scores[0]['rouge-2']['f']
rouge_l = scores[0]['rouge-l']['f']
rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3
rouge_avg

0.35490034990035496

# Q6. Average rouge score for all the data points

In [35]:
# rouge_score = Rouge()

def compute_rouge(record, rouge_score=Rouge()):
    answer_llm = record['answer_llm']
    answer_orig = record['answer_orig']

    scores = rouge_score.get_scores(answer_llm, answer_orig)
    
    rouge_1 = scores[0]['rouge-1']['f']
    rouge_2 = scores[0]['rouge-2']['f']
    rouge_l = scores[0]['rouge-l']['f']
    rouge_avg = (rouge_1 + rouge_2 + rouge_l) / 3

    return rouge_1, rouge_2, rouge_l, rouge_avg


In [36]:
from tqdm.auto import tqdm

rouge_1_scores = []
rouge_2_scores = []
rouge_l_scores = []
rouge_avg_scores = []

result_gpt4omini = df.to_dict(orient='records')

for record in tqdm(result_gpt4omini):
    rouge_1, rouge_2, rouge_l, rouge_avg = compute_rouge(record)
    rouge_1_scores.append(rouge_1)
    rouge_2_scores.append(rouge_2)
    rouge_l_scores.append(rouge_l)
    rouge_avg_scores.append(rouge_avg)

  0%|          | 0/300 [00:00<?, ?it/s]

In [37]:
df['rouge_1'] = rouge_1_scores
df['rouge_2'] = rouge_2_scores
df['rouge_l'] = rouge_l_scores
df['rouge_avg'] = rouge_avg_scores

In [38]:
df.head()

,answer_llm,answer_orig,document,question,course,cosine,rouge_1,rouge_2,rouge_l,rouge_avg
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp,0.506754,0.095238,0.028169,0.095238,0.072882
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp,0.388549,0.125000,0.055556,0.093750,0.091435
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp,0.718599,0.415584,0.177778,0.389610,0.327658
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp,0.337266,0.216216,0.047059,0.189189,0.150821
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp,0.521792,0.142076,0.033898,0.120219,0.098731


In [39]:
df['rouge_2'].describe()

count    300.000000
mean       0.206965
std        0.153550
min        0.000000
25%        0.097809
50%        0.178671
75%        0.286181
max        0.739130
Name: rouge_2, dtype: float64